In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from common_utilities import GlobalPaths

In [ ]:
# Initialize StockPrice Paths
StockPriceGlobalPath = GlobalPaths(source_name="DATA", object_name="StockPrice")
StockPrice_SILVER_LAYER_PATH = StockPriceGlobalPath.createLayer(layer_name="SILVER")
StockPrice_SILVER_FILE_PATH = StockPrice_SILVER_LAYER_PATH.joinpath("StockPrice_data.csv")

# Initialize Holdings Paths
HoldingsGlobalPath = GlobalPaths(source_name="DATA", object_name="Holdings")
Holdings_GOLD_LAYER_PATH = HoldingsGlobalPath.createLayer(layer_name="GOLD")
Holdings_GOLD_FILE_PATH = Holdings_GOLD_LAYER_PATH.joinpath("Holdings_data.csv")

# Initialize Investment Paths
InvestmentGlobalPath = GlobalPaths(source_name="DATA", object_name="Investment")
Investment_GOLD_LAYER_PATH = InvestmentGlobalPath.createLayer(layer_name="GOLD")
Investment_GOLD_FILE_PATH = Investment_GOLD_LAYER_PATH.joinpath("Investment_data.csv")

In [ ]:
## PRESENTATION LAYER
df_Holdings_GOLD = pd.read_csv(Holdings_GOLD_FILE_PATH)
df_Investment_GOLD = pd.read_csv(Investment_GOLD_FILE_PATH)
df_StockPrice_SILVER = pd.read_csv(StockPrice_SILVER_FILE_PATH)

# Convert datetime to date string
df_Holdings_GOLD["date"] = pd.to_datetime(df_Holdings_GOLD["date"])
df_Investment_GOLD["date"] = pd.to_datetime(df_Investment_GOLD["date"])
df_StockPrice_SILVER["date"] = pd.to_datetime(df_StockPrice_SILVER["date"])

# setting date column as index
df_Holdings_GOLD.set_index("date", inplace=True)
df_Investment_GOLD.set_index("date", inplace=True)
df_StockPrice_SILVER.set_index("date", inplace=True)

In [ ]:
# multiply
df_Investment_GOLD = df_Holdings_GOLD.multiply(df_Investment_GOLD)
# Add the sum column
df_Investment_GOLD["investment"] = df_Investment_GOLD.sum(axis=1)

# multiply
df_stock_value = df_Holdings_GOLD.multiply(df_StockPrice_SILVER)
# Add the sum column
df_stock_value["stock_value"] = df_stock_value.sum(axis=1)

In [ ]:
df = df_Investment_GOLD[["investment"]].join(df_stock_value[["stock_value"]])
df["pl"] = df["stock_value"] / df["investment"] * 100
df = df.round(2)
df = df[df["stock_value"] > 0]
df.info()

In [ ]:
# plot the graph
fig, ax = plt.subplots(figsize=(15, 5))

# plot individual lines
ax.plot(df.index, df["investment"], label="investment", color="steelblue")
ax.plot(df.index, df["stock_value"], label="stock_value", color="green")

# set major ticks to months and format the ticks
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
ax.xaxis.grid(True, which="major", linestyle=":")
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

# Add a grid to the y-axis
ax.yaxis.grid(True, which="major", linestyle="--")

# add axis labels and a title
plt.ylabel("Price")
plt.xlabel("Period")
plt.title("Buy Price vs. Stock Price")
ax.legend()

# display plot
plt.tight_layout()
plt.show()

In [ ]:
# plot the graph
fig, ax = plt.subplots(figsize=(15, 5))
ax.plot(df.index, df["pl"], label="profit_loss")

# set major ticks to months and format the ticks
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
ax.xaxis.grid(True, which="major", linestyle=":")
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

# Add a grid to the y-axis
ax.yaxis.grid(True, which="major", linestyle="--")

# add axis labels and a title
plt.ylabel("Percentage")
plt.xlabel("Period")
plt.title("Profit/Loss")
ax.legend()

# display plot
plt.tight_layout()
plt.show()